### Select patients for CFR model: Split patients in train, val and test sets ###

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
pet_data_dir = os.path.normpath('/mnt/obi0/phi/pet/pet_cfr')
meta_date = '200617'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200617


In [3]:
match_view_filename = 'pet_match365_diff_files_'+meta_date+'.parquet'
files_cfr = pd.read_parquet(os.path.join(meta_dir, match_view_filename))

print('Total number of patients      {}'.format(len(files_cfr.mrn.unique())))
print('Total number of echo studies  {}'.format(len(files_cfr.study.unique())))
print('Total number of PET studies   {}'.format(len(files_cfr.petmrn_identifier.unique())))
print('Total number of echos         {}'.format(len(files_cfr.filename.unique())))

files_cfr.head(2)

Total number of patients      3268
Total number of echo studies  6460
Total number of PET studies   3603
Total number of echos         307566


,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet,pet_measurement,difference(days),filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views
0,35156678,48b09010a2219aad_4903a582edf3bd118ffb3386065b,2018-10-15,2017-12-06,35156678_2018-10-15,-313,1.0,313.0,48b09010a2219aad_4903a582edf3bd118ffb3386065b_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,2017-12-06 13:11:41,48b09010a2219aad_4903a582edf3bd118ffb3386065b_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,47.769231,66.0,60.0,0.028951,0.028951,3.446593e-08,6.452001e-09,2.939033e-08,3.095016e-03,9.225302e-08,3.967397e-08,7.041133e-10,3.144316e-09,1.198068e-13,4.564848e-10,1.454091e-09,1.773068e-08,7.812481e-08,3.755003e-08,9.968956e-01,7.881907e-10,5.391607e-10,2.151894e-10,1.157160e-08,4.940982e-08,7.734493e-12,1.395856e-06,3.054052e-09,7.720808e-06,5.067335e-09,2.187016e-13,2.230961e-08,other,1.0
1,35156678,48b09010a2219aad_4903a582edf3bd118ffb3386065b,2018-10-15,2017-12-06,35156678_2018-10-15,-313,1.0,313.0,48b09010a2219aad_4903a582edf3bd118ffb3386065b_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09010a...,2017-12-06 13:11:41,48b09010a2219aad_4903a582edf3bd118ffb3386065b_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,20.087146,154.0,60.0,0.020448,0.020448,4.212547e-13,1.000000e+00,5.081462e-15,5.162780e-16,3.739126e-14,4.737708e-15,3.574842e-17,1.105575e-13,7.188538e-16,3.057809e-15,4.562448e-17,4.610617e-16,1.488447e-12,2.472197e-12,2.470155e-16,1.772980e-16,7.891783e-16,6.224149e-17,6.330659e-14,5.080497e-13,2.415044e-15,4.977857e-15,6.916603e-13,5.833864e-15,2.562479e-15,6.839081e-22,8.417050e-17,a2c_laocc,1.0


### Filter data sets: GLOBAL and NON-DEFECT variables ###

### Global variables ###

Notebook 3/17/2020: global_pet_cfr
File used: pet_cfr_petdata_02_26_2020_withperfandseg7.xlsx

As described above, 2871 after excluding CABG, transplant, and those with missing perfusion data
File used: post_2018_pets_with_clinical_cfr_all.csv

After excluding CABG, transplant, and missing CFR values, 167 PETs with any perfusion
Merge:

After combining, 3038 PETs
File saved as pets_with_echos_global_all.parquet
Use notes:

NOTE- there are petmrn_identifiers that have two rows- the one with post_2018==0 is the one to use, and the one with post_2018==1 should be excluded
Variables to use: rest_global_mbf, stress_global_mbf, global_cfr_calc
Other variables- myocardial_perfusion, segmental data, perfusion data, TID, gated SPECT results, calcium score, height, weight
For "cleaner" data, would exclude those with post-2018==1

#### Update 6/14/2020 ####

Created a revised version of the above that includes CABG cases (from pre-2018), and excludes post-2018 cases that are already in the pre-2018 dataset (n=7). Total 3718 rows. File at /mnt/obi0/phi/pet/pet_cfr/pets_with_echos_global_all_withcabg.parquet
Update 6/17/2020:

Created a revised version that has a column tracer_obi for the tracer used. Also, the 7 post-2018 duplicate cases are excluded so there are are 3031 studies.
File used- aiCohort_withPerfusion_addRadiopharm.xlsx
The ammonia cases before 7/25/2011 have the value 'listed as ammonia', and the rubidium cases after 7/25/2011 have the value 'listed as rubidium. For the remaining discrepant values (i.e. missing values, FDG, sestamibi), the tracer was assumed to be the tracer in use at the time.
Rubidium 1,740, ammonia 1,276, listed as ammonia 4, listed as rubidium 2
File saved as /mnt/obi0/phi/pet/pet_cfr/pets_with_echos_global_all_withtracer.parquet

In [4]:
global_pet_file = 'pets_with_echos_global_all_withtracer.parquet'
global_pet = pd.read_parquet(os.path.join(pet_data_dir, global_pet_file))
global_pet = global_pet.astype({'pet_date': 'datetime64[ns]'})
print(f'PET studies: {len(global_pet.petmrn_identifier.unique())}')
print(os.path.join(pet_data_dir, global_pet_file))
global_pet.head()

PET studies: 3031
/mnt/obi0/phi/pet/pet_cfr/pets_with_echos_global_all_withtracer.parquet


,mrn,pet_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,rest_global_mbf,stress_global_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,stress_seg17_mbf,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,height_in,weight_lb,reportID,subjectID,radiopharmaceutical,radiopharmaceutical2,tracer_obi
0,7924277,2008-08-15,7924277_2008-08-15,0,abnormal,1.368012,0.644,0.881,0.448,0.380,0.435,0.830,0.402,0.334,0.730,0.890,0.537,0.388,0.607,0.823,0.964,0.766,0.445,1.340,1.504,0.691,0.707,0.529,0.348,0.558,0.562,1.325,1.446,0.636,0.502,1.136,1.186,1.415,0.979,0.670,1.174,1.676,18.0,18.0,0.0,1.09,normal,NaN,67.0,133.0,69718.0,2540.0,Rubidium-82,None,rubidium
1,12853099,2006-01-25,12853099_2006-01-25,0,normal,2.109661,1.532,3.232,1.082,0.940,1.135,1.224,1.332,1.945,1.482,1.977,1.475,1.656,1.509,1.444,1.801,2.180,1.677,1.359,1.977,4.291,3.144,2.069,2.657,2.626,3.712,3.981,3.359,2.215,4.010,3.890,3.883,3.249,2.609,3.483,3.443,3.037,0.0,0.0,0.0,None,not assessed,0.0,64.0,160.0,69969.0,3379.0,Rubidium-82,None,rubidium
2,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.028,1.264,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,1.537,20.0,18.0,2.0,0.97,normal,6602.0,71.0,200.0,69993.0,3542.0,Rubidium-82,None,rubidium
3,12627030,2006-02-10,12627030_2006-02-10,0,abnormal,1.588915,1.732,2.752,1.984,1.499,1.556,1.333,1.134,1.292,2.499,1.883,1.703,1.794,1.873,1.933,1.767,2.056,1.662,1.804,1.754,2.940,0.664,0.640,2.197,1.413,3.395,4.533,1.301,2.149,2.391,3.300,4.714,3.184,2.567,4.154,4.662,3.595,16.0,0.0,16.0,0.79,normal,331.0,68.0,217.0,70008.0,2155.0,Rubidium-82,None,rubidium
4,15324312,2006-02-23,15324312_2006-02-23,0,normal,2.656655,1.465,3.892,0.945,0.813,0.892,0.970,1.401,1.217,1.614,1.580,1.303,1.957,1.879,1.794,1.421,1.773,1.677,1.891,1.846,2.559,2.759,2.654,2.434,3.313,3.502,4.081,4.340,3.978,4.170,4.426,4.944,4.343,4.143,4.748,4.544,4.328,0.0,0.0,0.0,0.94,normal,0.0,64.0,146.0,70012.0,3417.0,Rubidium-82,None,rubidium


In [5]:
global_pet[global_pet.petmrn_identifier=='1414556_2018-10-30']

,mrn,pet_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,rest_global_mbf,stress_global_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,stress_seg17_mbf,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,height_in,weight_lb,reportID,subjectID,radiopharmaceutical,radiopharmaceutical2,tracer_obi
2870,1414556,2018-10-30,1414556_2018-10-30,0,normal,1.809645,0.788,1.426,0.839083,0.799958,0.810667,0.721833,0.783875,0.781667,0.803083,0.918875,0.917708,0.872042,0.849458,0.767792,0.693972,0.796028,0.812972,0.736222,0.642786,1.4265,1.442625,1.5275,1.390125,1.45075,1.4535,1.446542,1.720542,1.705208,1.582792,1.645125,1.386583,1.128667,1.501056,1.447167,1.197389,1.21775,0.0,0.0,0.0,None,normal,145.0,63.0,189.0,166220.0,7521.0,N-13 Ammonia,None,ammonia


In [6]:
# Exclude files without frame_time
file_cfr_meta = files_cfr.loc[~files_cfr.frame_time.isnull()]

# Add echo data to the pet studies (inner join, to keep only keys in both dataframes)
global_pet_echo = global_pet.merge(file_cfr_meta, on = ['mrn', 'pet_date', 'petmrn_identifier'], how='inner')
print(f'PET studies:  {len(global_pet_echo.petmrn_identifier.unique())}')
print(f'ECHO studies: {len(global_pet_echo.study.unique())}')

PET studies:  2734
ECHO studies: 4525


In [7]:
print(f'All data:    patients:     {len(file_cfr_meta.mrn.unique())}')
print(f'All data:    PET studies:  {len(file_cfr_meta.petmrn_identifier.unique())}')
print(f'All data:    ECHO studies: {len(file_cfr_meta.study.unique())}')
print(f'All data:    videos:       {len(file_cfr_meta.filename.unique())}')
print()
print(f'Global data: patients:     {len(global_pet_echo.mrn.unique())}')
print(f'Global data: PET studies:  {len(global_pet_echo.petmrn_identifier.unique())}')
print(f'Global data: ECHO studies: {len(global_pet_echo.study.unique())}')
print(f'Global data: videos:       {len(global_pet_echo.filename.unique())}')

excluded_pet_studies = len(file_cfr_meta.petmrn_identifier.unique()) - len(global_pet_echo.petmrn_identifier.unique())
print(f'Excluded PET studies: {excluded_pet_studies}')

All data:    patients:     3257
All data:    PET studies:  3580
All data:    ECHO studies: 6288
All data:    videos:       307292

Global data: patients:     2591
Global data: PET studies:  2734
Global data: ECHO studies: 4525
Global data: videos:       217192
Excluded PET studies: 846


In [8]:
# Find out which echo studies do not have a4c views
# Get all studies WITH a4c views
a4c_study_list = list(global_pet_echo[global_pet_echo.max_view=='a4c'].study.unique())
print(len(a4c_study_list))

# Remove all studies from global list that have a4c views
global_pet_echo_no_a4c = global_pet_echo[~global_pet_echo.study.isin(a4c_study_list)]
print(f'Number of ECHO studies without a4c view: {len(global_pet_echo_no_a4c.study.unique())}')
print(f'For this number of patients:             {len(global_pet_echo_no_a4c.mrn.unique())}')
print(f'With this number of videos:              {len(global_pet_echo_no_a4c.filename.unique())}')

3926
Number of ECHO studies without a4c view: 599
For this number of patients:             507
With this number of videos:              18799


In [9]:
# Exclusions and filters
# A4C VIEW
global_pet_echo_a4c = global_pet_echo.loc[global_pet_echo.max_view=='a4c']
global_pet_echo_a4c = global_pet_echo_a4c.loc[~global_pet_echo_a4c.frame_time.isnull()]
global_a4c_pre18 = global_pet_echo_a4c.loc[global_pet_echo_a4c['post-2018']==0]

print(f'Global data: patients:     {len(global_pet_echo.mrn.unique())}')
print(f'Global data: PET studies:  {len(global_pet_echo.petmrn_identifier.unique())}')
print(f'Global data: ECHO studies: {len(global_pet_echo.study.unique())}')
print(f'Global data: videos:       {len(global_pet_echo.filename.unique())}')
print()
print('After a4c and post-2018 filters:')
print(f'Global data: patients:     {len(global_a4c_pre18.mrn.unique())}')
print(f'Global data: PET studies:  {len(global_a4c_pre18.petmrn_identifier.unique())}')
print(f'Global data: ECHO studies: {len(global_a4c_pre18.study.unique())}')
print(f'Global data: videos:       {len(global_a4c_pre18.filename.unique())}')
print()
print(f'Lost {len(global_pet_echo_a4c.filename.unique())-len(global_a4c_pre18.filename.unique())} due to post-2018 filtering.')

Global data: patients:     2591
Global data: PET studies:  2734
Global data: ECHO studies: 4525
Global data: videos:       217192

After a4c and post-2018 filters:
Global data: patients:     2287
Global data: PET studies:  2408
Global data: ECHO studies: 3681
Global data: videos:       11077

Lost 708 due to post-2018 filtering.


In [10]:
# Global_pet_echo table with the variables to use (drop rows with na in any of those variables)
global_pet_echo.head(2)
global_pet_variables_target = ['rest_global_mbf', 
                               'stress_global_mbf', 
                               'global_cfr_calc', 
                               'post-2018', 
                               'tracer_obi']

global_pet_variables = global_pet_variables_target.copy()
global_pet_variables.extend(list(files_cfr.columns))

global_pet_echo_variables = global_a4c_pre18[global_pet_variables].dropna(subset=global_pet_variables_target,
                                                                          axis=0)

print(f'Global data: patients:     {len(global_pet_echo_variables.mrn.unique())}')
print(f'Global data: PET studies:  {len(global_pet_echo_variables.petmrn_identifier.unique())}')
print(f'Global data: ECHO studies: {len(global_pet_echo_variables.study.unique())}')
print(f'Global data: videos:       {len(global_pet_echo_variables.filename.unique())}')
print(f'Tracer values:             {len(global_pet_echo_variables.tracer_obi.unique())}')

Global data: patients:     2287
Global data: PET studies:  2408
Global data: ECHO studies: 3681
Global data: videos:       11077
Tracer values:             4


In [11]:
# Complete list of unique petmrn_identifier
petmrn_identifier_list = list(global_pet.petmrn_identifier.unique())
petmrn_identifier_set = list(set(petmrn_identifier_list))

In [12]:
# Let's filter Rahuls list of missing echos
missing_echo = pd.read_parquet(os.path.join(meta_dir, 'mrn_pet_missing_echo_file.parquet'))
print(f'petmrn_identifier in original list: {len(missing_echo.petmrn_identifier.unique())}')
missing_echo.tail()
missing_echo_filtered = missing_echo[missing_echo.petmrn_identifier.isin(petmrn_identifier_set)].\
                        drop(columns=['pet_measurement'], axis=1).reset_index(drop=True)
print(f'petmrn_identifier in filtered list:   {len(missing_echo_filtered.petmrn_identifier.unique())}')
missing_no_echo_date = missing_echo_filtered.loc[missing_echo_filtered.echo_date.isnull()]
print(f'petmrn_identifier without echo dates: {len(missing_no_echo_date.petmrn_identifier.unique())}')

missing_echo_filtered_file = 'mrn_pet_missing_echo_file_filtered.parquet'
missing_echo_filtered.to_parquet(os.path.join(meta_dir, missing_echo_filtered_file))

petmrn_identifier in original list: 229
petmrn_identifier in filtered list:   72
petmrn_identifier without echo dates: 2


### Split the patients in train, validate and test sets ###
Although each view might have a little different patient pupulation distribution, because not all views are in each study. However, we want the same MRNS in each data set and for all views so that we can directly compare the performance of the algorithm for the same patients. We can expand the data frame above to add the splits.

In [13]:
def patientsplit(patient_list):

    train_test_split = 0.85
    train_eval_split = 0.90

    # Take a test set from all patients
    patient_list_train = np.random.choice(patient_list,
                                          size = int(np.floor(train_test_split*len(patient_list))),
                                          replace = False)
    patient_list_test = list(set(patient_list).difference(patient_list_train))
    train_test_intersection = set(patient_list_train).intersection(set(patient_list_test)) # This should be empty
    print('Intersection of patient_list_train and patient_list_test:', train_test_intersection)

    # Further separate some patients for evaluation from the training list
    patient_list_eval = np.random.choice(patient_list_train,
                                         size = int(np.ceil((1-train_eval_split)*len(patient_list_train))),
                                         replace = False)

    patient_list_train = set(patient_list_train).difference(patient_list_eval)
    train_eval_intersection = set(patient_list_train).intersection(set(patient_list_eval))
    print('Intersection of patient_list_train and patient_list_eval:', train_eval_intersection)

    # Show the numbers
    print('total patients:', len(patient_list))
    print()
    print('patients in set:', np.sum([len(patient_list_train),
                                     len(patient_list_eval),
                                     len(patient_list_test)]))
    print()
    print('patients in train:', len(patient_list_train))
    print('patients in eval:', len(patient_list_eval))
    print('patients in test:', len(patient_list_test))

    return patient_list_train, patient_list_eval, patient_list_test

In [14]:
dataset = global_pet_echo_variables
dataset_filename = 'global_pet_echo_dataset_'+meta_date+'_.parquet'
global_pet_echo_variables.head()

# Get a patient list
patient_list = list(dataset.sample(frac=1).mrn.unique())
patient_list_train, patient_list_eval, patient_list_test = patientsplit(patient_list)

patient_split = {'train': patient_list_train,
                 'eval': patient_list_eval,
                 'test': patient_list_test}

print('Patient IDs in train:', len(patient_split['train']))
print('Patient IDs in eval:', len(patient_split['eval']))
print('Patient IDs in test:', len(patient_split['test']))

print()

print('Intersection train-test:', set(patient_split['train']).intersection(set(patient_split['test'])))
print('Intersection train-eval:', set(patient_split['train']).intersection(set(patient_split['eval'])))
print('Intersection eval-test:', set(patient_split['eval']).intersection(set(patient_split['test'])))

Intersection of patient_list_train and patient_list_test: set()
Intersection of patient_list_train and patient_list_eval: set()
total patients: 2287

patients in set: 2287

patients in train: 1748
patients in eval: 195
patients in test: 344
Patient IDs in train: 1748
Patient IDs in eval: 195
Patient IDs in test: 344

Intersection train-test: set()
Intersection train-eval: set()
Intersection eval-test: set()


In [15]:
# Add mode column to files_cfr
split_list = []
for dset in patient_split.keys():
    dset_mrn_list = list(patient_split[dset])
    split_list.append(pd.DataFrame({'mrn': dset_mrn_list,
                                    'dset_mode': [dset]*len(dset_mrn_list)}))

split_df = pd.concat(split_list, ignore_index = True)

dataset_split = dataset.merge(right = split_df, on = 'mrn', how = 'left').\
                sample(frac = 1).\
                reset_index(drop = True)

In [16]:
print(f'TOTAL patients:     {len(dataset_split.mrn.unique())}')
print(f'TOTAL PET studies:  {len(dataset_split.petmrn_identifier.unique())}')
print(f'TOTAL ECHO studies: {len(dataset_split.study.unique())}')
print(f'TOTAL videos:       {len(dataset_split.filename.unique())}')

dset_list = ['train', 'eval', 'test']
for dset in dset_list:
    df_dset = dataset_split[dataset_split.dset_mode==dset]
    print()
    print(f'patients in {dset}:     {len(df_dset.mrn.unique())}')
    print(f'PET studies in {dset}:  {len(df_dset.petmrn_identifier.unique())}')
    print(f'ECHO studies in {dset}: {len(df_dset.study.unique())}')
    print(f'videos in {dset}:       {len(df_dset.filename.unique())}')

TOTAL patients:     2287
TOTAL PET studies:  2408
TOTAL ECHO studies: 3681
TOTAL videos:       11077

patients in train:     1748
PET studies in train:  1844
ECHO studies in train: 2859
videos in train:       8562

patients in eval:     195
PET studies in eval:  204
ECHO studies in eval: 316
videos in eval:       958

patients in test:     344
PET studies in test:  360
ECHO studies in test: 506
videos in test:       1557


In [17]:
# Let's look at the video tracer numbers
tracer_list = dataset_split.tracer_obi.unique()
print(tracer_list)
for tracer in tracer_list:
    df_tracer = dataset_split[dataset_split.tracer_obi==tracer]
    for dset in dset_list:
        df_dset = df_tracer[df_tracer.dset_mode==dset]
        print()
        print(f'{dset}: patients for {tracer}:     {len(df_dset.mrn.unique())}')
        print(f'{dset}: PET studies for {tracer}:  {len(df_dset.petmrn_identifier.unique())}')
        print(f'{dset}: ECHO studies for {tracer}: {len(df_dset.study.unique())}')
        print(f'{dset}: videos for {tracer}:       {len(df_dset.filename.unique())}')

['ammonia' 'rubidium' 'listed as ammonia' 'listed as rubidium']

train: patients for ammonia:     714
train: PET studies for ammonia:  738
train: ECHO studies for ammonia: 1247
train: videos for ammonia:       4131

eval: patients for ammonia:     80
eval: PET studies for ammonia:  81
eval: ECHO studies for ammonia: 139
eval: videos for ammonia:       473

test: patients for ammonia:     150
test: PET studies for ammonia:  158
test: ECHO studies for ammonia: 247
test: videos for ammonia:       808

train: patients for rubidium:     1064
train: PET studies for rubidium:  1101
train: ECHO studies for rubidium: 1605
train: videos for rubidium:       4412

eval: patients for rubidium:     119
eval: PET studies for rubidium:  123
eval: ECHO studies for rubidium: 177
eval: videos for rubidium:       485

test: patients for rubidium:     198
test: PET studies for rubidium:  201
test: ECHO studies for rubidium: 258
test: videos for rubidium:       748

train: patients for listed as ammonia:   

In [18]:
dataset_split.columns

Index(['rest_global_mbf', 'stress_global_mbf', 'global_cfr_calc', 'post-2018', 'tracer_obi', 'mrn', 'study', 'pet_date', 'echo_date', 'petmrn_identifier', 'days_post_pet', 'pet_measurement', 'difference(days)', 'filename', 'dir', 'datetime', 'fileid', 'institution', 'model', 'manufacturer', 'index', 'frame_time', 'number_of_frames', 'heart_rate', 'deltaX', 'deltaY', 'a2c', 'a2c_laocc', 'a2c_lvocc_s', 'a3c', 'a3c_laocc', 'a3c_lvocc_s', 'a4c', 'a4c_far', 'a4c_laocc', 'a4c_lvocc_s', 'a4c_rv', 'a4c_rv_laocc', 'a5c', 'apex', 'other', 'plax_far', 'plax_lac', 'plax_laz', 'plax_laz_ao', 'plax_plax', 'psax_avz', 'psax_az', 'psax_mv', 'psax_pap', 'rvinf', 'subcostal', 'suprasternal', 'max_view', 'sum_views', 'dset_mode'], dtype='object')

In [19]:
# Prepare the final set that we will use for writing TFR files. We don't want any rows with NAs in some columns.
col_set = ['rest_global_mbf', 'rest_global_mbf', 'global_cfr_calc', 'tracer_obi', 
           'pet_measurement', 'sum_views', 'dset_mode']
dataset_split_tfr = dataset_split.dropna(subset = col_set, axis=0)

print('Dropped {} rows.'.format(dataset_split.shape[0]-dataset_split_tfr.shape[0]))

# Add some other information that we need and shuffle the whole thing
dataset_split_tfr = dataset_split_tfr.assign(rate = np.round(1/dataset_split_tfr.frame_time*1e3, decimals = 1))
dataset_split_tfr = dataset_split_tfr.assign(dur = dataset_split_tfr.frame_time*1e3*dataset_split_tfr.number_of_frames)

dataset_split_tfr = dataset_split_tfr.sample(frac = 1)

dataset_split.loc[~dataset_split.index.isin(dataset_split_tfr.index)]

Dropped 0 rows.


,rest_global_mbf,stress_global_mbf,global_cfr_calc,post-2018,tracer_obi,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet,pet_measurement,difference(days),filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views,dset_mode


In [26]:
print(list(dataset_split_tfr.tracer_obi.unique()))

['rubidium', 'ammonia', 'listed as ammonia', 'listed as rubidium']


In [20]:
dataset_split_tfr.to_parquet(os.path.join(meta_dir, dataset_filename))
print('Saved to file:', dataset_filename)
print(dataset_split_tfr.shape)
dataset_split_tfr.head()

Saved to file: global_pet_echo_dataset_200617_.parquet
(11077, 58)


,rest_global_mbf,stress_global_mbf,global_cfr_calc,post-2018,tracer_obi,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet,pet_measurement,difference(days),filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,sum_views,dset_mode,rate,dur
1649,2.226,1.803,0.809973,0,rubidium,5229976,4b7e4b6bb89c174b_4903a44ab097f2f3395f713f7609,2009-10-27,2009-10-26,5229976_2009-10-27,-1,1.0,1.0,4b7e4b6bb89c174b_4903a44ab097f2f3395f713f7609_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7e/4b7e4b6bb...,2009-10-26 14:37:44,4b7e4b6bb89c174b_4903a44ab097f2f3395f713f7609_...,BWH,iE33,Philips Medical Systems,0.0,33.333,52.0,69.0,0.037555,0.037555,3.103878e-15,5.590010e-16,9.579883e-17,1.032404e-16,5.882793e-19,4.117114e-17,1.000000,1.749426e-16,1.353042e-17,4.129842e-16,1.813307e-11,9.411450e-17,3.009333e-16,7.853347e-15,2.656639e-16,4.367141e-17,4.527414e-15,1.390832e-16,4.485841e-17,5.766664e-16,2.515440e-14,6.059152e-17,3.221223e-16,2.344755e-14,2.036121e-18,1.812663e-19,1.373915e-17,a4c,1.0,eval,30.0,1733316.0
7974,0.726,0.495,0.681818,0,rubidium,23686975,4900481b3301b2c6_4903a44ab12c9a234c5d030df1ce,2010-01-12,2009-07-17,23686975_2010-01-12,-179,1.0,179.0,4900481b3301b2c6_4903a44ab12c9a234c5d030df1ce_...,/mnt/obi0/phi/echo/npyFiles/BWH/4900/4900481b3...,2009-07-17 13:29:44,4900481b3301b2c6_4903a44ab12c9a234c5d030df1ce_...,BWH,iE33,Philips Medical Systems,0.0,33.333,60.0,50.0,0.048411,0.048411,8.507612e-11,1.793651e-08,9.698853e-11,2.371943e-10,9.562035e-13,5.033167e-11,0.999995,8.764711e-10,1.859114e-09,1.209920e-11,1.088010e-08,4.697926e-12,5.319416e-09,8.357802e-09,4.093921e-10,4.197729e-12,1.399132e-11,3.415286e-10,1.251839e-11,2.317613e-09,1.078508e-08,5.043116e-06,2.798837e-11,1.251374e-07,4.845703e-09,1.247392e-11,6.879416e-08,a4c,1.0,train,30.0,1999980.0
4131,0.642,0.951,1.481308,0,ammonia,7254246,4b7c2c7434781c7a_4903a582ec77f16c8f7bb73215af,2017-07-20,2017-06-21,7254246_2017-07-20,-29,1.0,29.0,4b7c2c7434781c7a_4903a582ec77f16c8f7bb73215af_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7c/4b7c2c743...,2017-06-21 14:14:23,4b7c2c7434781c7a_4903a582ec77f16c8f7bb73215af_...,BWH,iE33,Philips Medical Systems,0.0,33.333,49.0,55.0,0.065784,0.065784,1.464292e-12,1.006150e-13,1.662668e-13,1.565471e-12,2.322911e-11,1.537790e-10,1.000000,5.193972e-12,6.906591e-11,6.772417e-13,1.873226e-09,5.956266e-13,4.335831e-11,1.148705e-12,1.941967e-09,1.854237e-12,2.685623e-08,1.796025e-13,7.556777e-13,1.351569e-11,4.497765e-10,2.993599e-10,2.469222e-13,4.896105e-11,1.248502e-08,4.069121e-15,1.111339e-12,a4c,1.0,eval,30.0,1633317.0
1786,0.654,1.140,1.743119,0,ammonia,13411699,4a1532536642d525_4903a58359a27e0e5fcef587627f,2016-11-14,2016-10-03,13411699_2016-11-14,-42,1.0,42.0,4a1532536642d525_4903a58359a27e0e5fcef587627f_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a15/4a1532536...,2016-10-03 10:17:03,4a1532536642d525_4903a58359a27e0e5fcef587627f_...,BWH,iE33,Philips Medical Systems,0.0,33.333,47.0,49.0,0.035327,0.035327,2.130290e-06,6.650966e-06,6.564918e-11,1.499423e-11,1.235822e-11,7.734752e-09,0.999803,3.106802e-11,8.341503e-12,3.053812e-09,1.864053e-04,2.875095e-10,1.229904e-06,7.124085e-11,8.938346e-08,5.397026e-10,2.525883e-07,2.856190e-09,3.795014e-11,1.454968e-08,1.333803e-07,1.546846e-07,1.455377e-10,1.133365e-09,9.378541e-10,5.806875e-14,9.183303e-12,a4c,1.0,train,30.0,1566651.0
6255,1.644,2.002,1.217762,0,rubidium,24722068,4907be24837021b4_4903a58584283b89d98db46b463a,2010-06-14,2010-06-16,24722068_2010-06-14,2,1.0,2.0,4907be24837021b4_4903a58584283b89d98db46b463a_...,/mnt/obi0/phi/echo/npyFiles/BWH/4907/4907be248...,2010-06-16 15:26:14,4907be24837021b4_4903a58584283b89d98db46b463a_...,BWH,iE33,Philips Medical Systems,0.0,33.333,34.0,100.0,0.039451,0.039451,2.277

In [21]:
minframes = 40
max_frame_time = 33.34
print('Minimum frames:   {}'.format(minframes))
print('Maximum frame_time: {}'.format(max_frame_time))
minduration = max_frame_time*minframes
print('Minimum duration: {}'.format(minduration))

#minframes = int(np.ceil(minrate*minduration))

maxrows = dataset_split_tfr.shape[0]
dataset_disqualified = dataset_split_tfr[(dataset_split_tfr.frame_time > max_frame_time) | (dataset_split_tfr.dur < minduration)]
dataset_qualified = dataset_split_tfr[(dataset_split_tfr.frame_time <= max_frame_time) & (dataset_split_tfr.dur >= minduration)]

n_videos = len(dataset_split_tfr.filename.unique())
n_disqualified = len(dataset_disqualified.filename.unique())
n_qualified = len(dataset_qualified.filename.unique())

print()
print('Total videos: {}'.format(n_videos))
print('Disqualified videos {}, fraction:{:.1f}'.format(n_disqualified, 
                                                     np.round(n_disqualified/n_videos*100), decimals=1))
print('Qualified videos {}, fraction:{:.1f}'.format(n_qualified, 
                                                     np.round(n_qualified/n_videos*100), decimals=1))

Minimum frames:   40
Maximum frame_time: 33.34
Minimum duration: 1333.6000000000001

Total videos: 11077
Disqualified videos 2502, fraction:23.0
Qualified videos 8575, fraction:77.0


In [24]:
# Report train val test numbers that qualify....
tracer_list = dataset_split.tracer_obi.unique()
print(tracer_list)
for dset in dset_list:
    df_dset = dataset_qualified[dataset_qualified.dset_mode==dset]
    print()
    print(f'{dset}: patients for TOTAL:     {len(df_dset.mrn.unique())}')
    print(f'{dset}: PET studies for TOTAL:  {len(df_dset.petmrn_identifier.unique())}')
    print(f'{dset}: ECHO studies for TOTAL: {len(df_dset.study.unique())}')
    print(f'{dset}: videos for TOTAL:       {len(df_dset.filename.unique())}')
    
    for tracer in tracer_list:
        df_tracer = df_dset[df_dset.tracer_obi==tracer]
        print()
        print(f'{dset}: patients for {tracer}:     {len(df_dset.mrn.unique())}')
        print(f'{dset}: PET studies for {tracer}:  {len(df_dset.petmrn_identifier.unique())}')
        print(f'{dset}: ECHO studies for {tracer}: {len(df_dset.study.unique())}')
        print(f'{dset}: videos for {tracer}:       {len(df_dset.filename.unique())}')

['ammonia' 'rubidium' 'listed as ammonia' 'listed as rubidium']

train: patients for TOTAL:     1448
train: PET studies for TOTAL:  1512
train: ECHO studies for TOTAL: 2166
train: videos for TOTAL:       6661

train: patients for ammonia:     1448
train: PET studies for ammonia:  1512
train: ECHO studies for ammonia: 2166
train: videos for ammonia:       6661

train: patients for rubidium:     1448
train: PET studies for rubidium:  1512
train: ECHO studies for rubidium: 2166
train: videos for rubidium:       6661

train: patients for listed as ammonia:     1448
train: PET studies for listed as ammonia:  1512
train: ECHO studies for listed as ammonia: 2166
train: videos for listed as ammonia:       6661

train: patients for listed as rubidium:     1448
train: PET studies for listed as rubidium:  1512
train: ECHO studies for listed as rubidium: 2166
train: videos for listed as rubidium:       6661

eval: patients for TOTAL:     166
eval: PET studies for TOTAL:  174
eval: ECHO studies for